In [1]:
!pwd
# os.chdir('../')
# !pwd

from glob import glob
from matplotlib import pyplot as plt
import numpy as np
import os
from skimage.io import imread
import numpy as np
import pandas as pd
import df_utils
from pytorch_lightning import seed_everything
from sklearn.model_selection import train_test_split

/home/genrev_kiel_hizon/SAR_flood_segmentation/src


## Generate Annotations CSV

In [2]:
seed_everything(42, workers=True)

Global seed set to 42


42

In [3]:
train_df = df_utils.create_df('../data/train/train/')
train_df.head()

,vv_image_path,vh_image_path,flood_label_path,water_body_label_path,region
0,../data/train/train/bangladesh_20170314t115609...,../data/train/train/bangladesh_20170314t115609...,../data/train/train/bangladesh_20170314t115609...,../data/train/train/bangladesh_20170314t115609...,bangladesh
1,../data/train/train/bangladesh_20170314t115609...,../data/train/train/bangladesh_20170314t115609...,../data/train/train/bangladesh_20170314t115609...,../data/train/train/bangladesh_20170314t115609...,bangladesh
2,../data/train/train/bangladesh_20170314t115609...,../data/train/train/bangladesh_20170314t115609...,../data/train/train/bangladesh_20170314t115609...,../data/train/train/bangladesh_20170314t115609...,bangladesh
3,../data/train/train/bangladesh_20170314t115609...,../data/train/train/bangladesh_20170314t115609...,../data/train/train/bangladesh_20170314t115609...,../data/train/train/bangladesh_20170314t115609...,bangladesh
4,../data/train/train/bangladesh_20170314t115609...,../data/train/train/bangladesh_20170314t115609...,../data/train/train/bangladesh_20170314t115609...,../data/train/train/bangladesh_20170314t115609...,bangladesh


In [4]:
val_df = df_utils.create_df('../data/val_with_ref_labels/test')
val_df.head()

,vv_image_path,vh_image_path,flood_label_path,water_body_label_path,region
0,../data/val_with_ref_labels/test/florence_2018...,../data/val_with_ref_labels/test/florence_2018...,../data/val_with_ref_labels/test/florence_2018...,../data/val_with_ref_labels/test/florence_2018...,florence
1,../data/val_with_ref_labels/test/florence_2018...,../data/val_with_ref_labels/test/florence_2018...,../data/val_with_ref_labels/test/florence_2018...,../data/val_with_ref_labels/test/florence_2018...,florence
2,../data/val_with_ref_labels/test/florence_2018...,../data/val_with_ref_labels/test/florence_2018...,../data/val_with_ref_labels/test/florence_2018...,../data/val_with_ref_labels/test/florence_2018...,florence
3,../data/val_with_ref_labels/test/florence_2018...,../data/val_with_ref_labels/test/florence_2018...,../data/val_with_ref_labels/test/florence_2018...,../data/val_with_ref_labels/test/florence_2018...,florence
4,../data/val_with_ref_labels/test/florence_2018...,../data/val_with_ref_labels/test/florence_2018...,../data/val_with_ref_labels/test/florence_2018...,../data/val_with_ref_labels/test/florence_2018...,florence


## Find images with significant flooding

In [5]:
train_df['has_mask'] = train_df['flood_label_path'].apply(df_utils.has_mask)

In [6]:
val_df['has_mask'] = val_df['flood_label_path'].apply(df_utils.has_mask)

In [7]:
train_df['has_mask'].value_counts()

False    22471
True     10934
Name: has_mask, dtype: int64

In [8]:
val_df['has_mask'].value_counts()

False    7290
True     3110
Name: has_mask, dtype: int64

## Find images where the satellite image is binary (invalid)

In [9]:
train_df['invalid'] = (train_df['vv_image_path'].apply(df_utils.remove_binary))
val_df['invalid'] = (val_df['vv_image_path'].apply(df_utils.remove_binary))

In [10]:
train_df[(train_df['invalid']==True) & (train_df['has_mask']==True)].sample(10)

,vv_image_path,vh_image_path,flood_label_path,water_body_label_path,region,has_mask,invalid
11697,../data/train/train/nebraska_20170520t002113/t...,../data/train/train/nebraska_20170520t002113/t...,../data/train/train/nebraska_20170520t002113/t...,../data/train/train/nebraska_20170520t002113/t...,nebraska,True,True
18173,../data/train/train/nebraska_20171222t002118/t...,../data/train/train/nebraska_20171222t002118/t...,../data/train/train/nebraska_20171222t002118/t...,../data/train/train/nebraska_20171222t002118/t...,nebraska,True,True
7843,../data/train/train/nebraska_20170213t002121/t...,../data/train/train/nebraska_20170213t002121/t...,../data/train/train/nebraska_20170213t002121/t...,../data/train/train/nebraska_20170213t002121/t...,nebraska,True,True
13171,../data/train/train/nebraska_20170601t002114/t...,../data/train/train/nebraska_20170601t002114/t...,../data/train/train/nebraska_20170601t002114/t...,../data/train/train/nebraska_20170601t002114/t...,nebraska,True,True
9220,../data/train/train/nebraska_20170402t002111/t...,../data/train/train/nebraska_20170402t002111/t...,../data/train/train/nebraska_20170402t002111/t...,../data/train/train/nebraska_20170402t002111/t...,nebraska,True,True
17889,../data/train/train/nebraska_20171222t002118/t...,../data/train/train/nebraska_20171222t002118/t...,../data/train/train/nebraska_20171222t002118/t...,../data/train/train/nebraska_20171222t002118/t...,nebraska,True,True
7751,../data/train/train/nebraska_20170213t002121/t...,../data/train/train/nebraska_20170213t002121/t...,../data/train/train/nebraska_20170213t002121/t...,../data/train/train/nebraska_20170213t002121/t...,nebraska,True,True
7939,../data/train/train/nebraska_20170213t002121/t...,../data/train/train/nebraska_20170213t002121/t...,../data/train/train/nebraska_20170213t002121/t...,../data/train/train/nebraska_20170213t002121/t...,nebraska,True,True
17127,../data/train/train/nebraska_20171210t002119/t...,../data/train/train/nebraska_20171210t002119/t...,../data/train/train/nebraska_20171210t002119/t...,../data/train/train/nebraska_20171210t002119/t...,nebraska,True,True
10689,../data/train/train/nebraska_20170508t002113/t...,../data/train/train/nebraska_20170508t002113/t...,../data/train/train/nebraska_20170508t002113/t...,../data/train/train/nebraska_20170508t002113/t...,nebraska,True,True


## Remove Invalid

In [11]:
train_df = train_df[train_df['invalid']!=True]
val_df = val_df[val_df['invalid']!=True]

## Split val -> val, test

In [12]:
val, test = train_test_split(val_df, stratify=val_df['has_mask'], test_size=0.5)

## Save final product to CSV

In [13]:
train_df.to_csv('../data/train.csv')
val.to_csv('../data/val.csv')
test.to_csv('../data/test.csv')

## Exploration

In [14]:
print(f'Train: {train_df.shape}')
print(f'Val: {val.shape}')
print(f'Test: {test.shape}')

Train: (25021, 7)
Val: (3306, 7)
Test: (3307, 7)


In [15]:
print(f'Train: {train_df[train_df.has_mask==True].shape}')
print(f'Val: {val[val.has_mask==True].shape}')
print(f'Test: {test[test.has_mask==True].shape}')

Train: (10298, 7)
Val: (1546, 7)
Test: (1547, 7)
